In [8]:
import boto3
import os
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

client = boto3.client('s3')

bucket = "ml-for-bem"
# experiment_name = "single-climate-zone/v1/nyc"
# experiment_name = "full_climate_zone/v1/train"
# experiment_name = "full_climate_zone/v2/test"
# experiment_name = "full_climate_zone/v3/train"
# experiment_name = "full_climate_zone/v3/test"
# experiment_name = "full_climate_zone/v4/train"
# experiment_name = "full_climate_zone/v4/test"
# experiment_name = "full_climate_zone/v5/train"
# experiment_name = "full_climate_zone/batch-test/train"
# experiment_name = "full_climate_zone/v6/test"
# experiment_name = "single-climate-zone/v2/train"
# experiment_name = "full_climate_zone/v7/train"
# experiment_name = "full_climate_zone/v7/test"
# experiment_name = "single-climate-zone/toronto/v7/test"
experiment_name = "full_climate_zone/v8/train"

In [9]:
paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket, Prefix=experiment_name + "/errors/")

error_ids = []
files = []
for page in pages:
    try:
        contents = page["Contents"]
    except KeyError:
        print("No errors reported!")
    else:
        for obj in page['Contents']:
            files.append(obj['Key'])
            error_ids.append(obj['Key'].split('/')[-1].split('.')[0])
        
print(len(error_ids))

70


In [12]:
# get a list of all the files in the /monthly/ folder

pages = paginator.paginate(Bucket=bucket, Prefix=experiment_name + "/monthly/")

monthly_files = []
for page in pages:
    for obj in page['Contents']:
        monthly_files.append(obj['Key'])
print(len(monthly_files))

5000


In [13]:
def download_and_open(file):
    client.download_file(bucket, file, "data/hdf5/" + file)
    with pd.HDFStore("data/hdf5/" + file) as store:
        df = store["batch_results"]
    os.remove("data/hdf5/" + file)
    return df

In [18]:
# use a thread pool executor to download all files in monthly_files
os.makedirs("data/hdf5/" + experiment_name + "/monthly", exist_ok=True)
with ThreadPoolExecutor(max_workers=8) as executor:
    dfs = list(tqdm(executor.map(download_and_open, monthly_files), total=len(monthly_files)))

dfs = pd.concat(dfs, axis=0)
print(len(dfs))


  0%|          | 0/5000 [00:00<?, ?it/s]

1000000


In [19]:
# dfs = []

# # download all of the files in the monthly folder to data/{experiment_name}
# os.makedirs("data/hdf5/" + experiment_name + "/monthly", exist_ok=True)
# i = 0
# for file in tqdm(monthly_files):
#     client.download_file(bucket, file, "data/hdf5/" + file)
#     with pd.HDFStore("data/hdf5/" + file) as store:
#         df = store["batch_results"]
#         dfs.append(df)
#     i = i + 1
#     os.remove("data/hdf5/" + file)

# dfs = pd.concat(dfs, axis=0)
# print(len(dfs))

In [20]:
ids = dfs.index.get_level_values("id")    
error_mask = ids.isin(error_ids)
dfs = dfs.set_index(error_mask,  append=True, )
dfs.index.names = dfs.index.names[:-1] + ["error"]
dfs.index.to_frame(index=False).head()
dfs.to_hdf("data/hdf5/" + experiment_name + "/monthly.hdf", key="batch_results", mode="w")

In [21]:
# upload the monthly.hdf file to s3
client.upload_file("data/hdf5/" + experiment_name + "/monthly.hdf", bucket, experiment_name + "/monthly.hdf")

In [23]:
# Concatenate with another df if desired
# dfs_7 = pd.read_hdf("data/lightning/full_climate_zone/v7/train/monthly.hdf", key="batch_results")
# df_all = pd.concat([dfs, dfs_7], axis=0)
# df_all.head()
# df_all.to_hdf("data/hdf5/" + experiment_name + "/monthly.hdf", key="batch_results", mode="w")
# client.upload_file("data/hdf5/" + experiment_name + "/monthly.hdf", bucket, experiment_name + "/monthly.hdf")
# len(df_all)

KeyValue                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Core  \
Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Heating   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             2018-01-01   
id                                   climate_zone base_epw width    height   perim_depth core_depth roof_2_footprint ground_2_footprint orientation wwr      HeatingSetpoint CoolingSetpoint LightingPowerDensity EquipmentPowerDensity PeopleDensity Infiltration VentilationPerArea VentilationPerPerson VentilationMode FacadeMass RoofMass FacadeRValue RoofRValue SlabRValue WindowUValue WindowShgc EconomizerSettings RecoverySettings schedules_seed shading_0 shading_1 shading_2 shading_3 shading_4 shading_5 shading_6 shading_7 shading_8 shading_9 shading_10 shading_11 error              
7c5821ab-175b-41cc-856a-5d87e2c958c8 16           453      3.152378 5.651718 8.649273    51.174243  0.047189         0.548744           0.310904    0.380062 21.183443       22.665220       26.752688            15.612595             0.473368      0.000317     0.003569           0.002158             1               3          3        3.478637     5.448976   8.752302   3.506175     0.231680   1                  0                15956915       1.238232  0.536065  1.444018  1.200531  0.862240  0.624791  1.516253  0.619734  0.855885  0.588814  1.354809   1.078180   False  71.415218   
ef7098e7-5adc-4512-b364-9081b649757b 5            518      2.102422 4.131947 7.311728    28.514648  0.448194         0.572528           0.360122    0.859487 23.961971       29.267478       22.473581            32.996386             0.338127      0.000554     0.001741           0.000364             1               2          3        9.123763     16.639477  6.101716   1.717597     0.356072   1                  0                5135319        0.937088  0.891534  1.298100  0.277938  0.256823  0.971682  0.043852  1.522131  0.918336  0.509806  1.439592   0.323061   False   0.000000   
f547f14c-674c-46b9-b7b8-9daf40b82890 3            16       2.650632 4.997418 4.668848    45.443843  0.322659         0.930347           2.663827    0.301595 19.703825       25.268049       9.800096             6.901501              0.234178      0.000629     0.001817           0.007785             1               1          2        14.325341    10.064605  9.673426   2.105162     0.825889   0                  0                787542         0.602160  0.065212  0.512904  1.3